In [24]:
# Acknowledgments: https://medium.com/c%C3%B3digo-ecuador/how-to-python-web-scrape-the-nasdaq-stock-ex-dividend-calendar-648b6063c659

import pandas as pd
import requests, datetime, calendar

In [25]:
# company tickers of interest

tickers = ["AAPL", "MSFT", "T"]

In [29]:
# Displays dividend for the current month. 
# If the dataframe returned is empty it means the companies(in the tickers list) didn't pay 
# dividend for the current month

class dividend_calendar:
    #class attributes
    calendars = []
    url = 'https://api.nasdaq.com/api/calendar/dividends'
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'DNT': "1",
        'Origin': 'https://www.nasdaq.com/',
        'Sec-Fetch-Mode': 'cors',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0)'
    }
    
    def __init__(self, year, month):
        '''
          Parameters
          -------
          year : year int
          month : month int

          Returns
          -------
          sets the instance attributes for the year and month of the object
        '''
        self.year = int(year)
        self.month = int(month)
    
    def date_str(self, day):
        '''
          Parameters
          ----------
          day int
          
          Returns 
          --------
          date string in the ‘%Y-%m-%d’ format 
        '''
        date_obj = datetime.date(self.year, self.month, day)
        return date_obj.strftime(format='%Y-%m-%d')
    
    def scraper(self, date_str):
        ''' 
          Scrapes JSON object from page using requests module

          Parameters
          - - - - - 
          url : URL string
          hdrs : Header information
          date_str: string in yyyy-mm-dd format

          Returns
          - - - -
          dictionary : Returns a JSON dictionary at a given URL.
         
        '''
        params = {'date': date_str}
        page = requests.get(self.url, headers=self.headers, params=params)
        dictionary = page.json()
        return dictionary  
    
    def dict_to_df(self, cal_dict):
        
        ''' 
          Converts the JSON dictionary into a pandas dataframe
          Appends the dataframe to calendars class attribute
          Parameters
          ----------
          cal_dict : Output from the scraper method as input.

          Returns
          -------
          calendar : Dataframe of stocks with that exdividend date

          Will append a dataframe to the calendars list (class 
          attribute). Otherwise, it will return an empty dataframe.         
        '''
        rows = cal_dict.get('data').get('calendar').get('rows')
        calendar = pd.DataFrame(rows)
        self.calendars.append(calendar)
        return calendar
    def calendar(self, day):
        '''
          Combines the scrape and dict_to_df methods
          
          Parameters
          ----------
          day : day of the month as string or number.
          
          Returns
          -------
          dictionary : Returns a JSON dictionary with keys 
          dictionary.keys() => data, message, status
          
          Next Levels: 
          dictionary['data'].keys() => calendar, timeframe
          dictionary['data']['calendar'].keys() => headers, rows
          dictionary['data']['calendar']['headers'] => column names
          dictionary['data']['calendar']['rows'] => dictionary list
    
        '''
        day = int(day)
        date_str = self.date_str(day)
        dictionary = self.scraper(date_str)
        self.dict_to_df(dictionary)
        return dictionary

if __name__ == '__main__':
#Edit year and month below as per your requirements
    year = datetime.datetime.now().year
    month = datetime.datetime.now().month
    print(f"Getting dividend info for {datetime.date(1900, month, 1).strftime('%B')} {year}")
    days_in_month = calendar.monthrange(year, month)[1]
    current_month = dividend_calendar(year, month)
#define lambda function to iterate over list of days     
    function = lambda days: current_month.calendar(days)
    
#define list of ints between 1 and the number of days in the month
    iterator = list(range(1, days_in_month+1))
#Scrape calendar for each day of the month                    
    objects = list(map(function, iterator))
#concatenate all the calendars in the class attribute
    concat_df = pd.concat(current_month.calendars)
    
#Drop any rows with missing data
    drop_df = concat_df.dropna(how='any')
    
#set the dataframe's row index to the company name
    final_df = drop_df.set_index('companyName')
    mylist = final_df.loc[final_df['symbol'].isin(tickers)]
    print(mylist)

Getting dividend info for October 2021
            symbol dividend_Ex_Date payment_Date record_Date  dividend_Rate  \
companyName                                                                   
AT&T Inc.        T       10/07/2021   11/01/2021  10/11/2021           0.52   

             indicated_Annual_Dividend announcement_Date  
companyName                                               
AT&T Inc.                         2.08        09/23/2021  
